In [1]:
try:
    from tqdm import tqdm
except ImportError:
    def tqdm(iterator, *args, **kwargs):
        return iterator
import os, sys, glob, warnings, glob    
import numpy as np
import scipy as sp
from scipy import constants
from pylab import *
import joblib
import importlib
from importlib import reload
sys.path.append("/home/owhgabri/My_GitHub/MATHUSLA_JupyterNoteBooks/tracker")
sys.path.insert(1, "/home/owhgabri/My_GitHub/MATHUSLA_JupyterNoteBooks/tracker")
os.chdir('/home/owhgabri/My_GitHub/pyTracker')
print(os.getcwd())
print(joblib.__version__)

/home/owhgabri/My_GitHub/pyTracker
1.4.2


In [2]:
import scipy
import copy as cp

# ROOT
import ROOT as root

# Matplotlib
import matplotlib.pyplot as plt
from matplotlib import collections, colors, transforms

%matplotlib inline
%config InlineBackend.figure_format='retina'
# %matplotlib widget

import pprint

Welcome to JupyROOT 6.26/14


In [3]:
import kalmanfilter as KF
import utilities as Util
import trackfinder as TF
import vertexfinder as VF
import datatypes
from datatypes import *

reload(TF)
reload(VF)
reload(Util)

<module 'utilities' from '/home/owhgabri/My_GitHub/MATHUSLA_JupyterNoteBooks/tracker/utilities.py'>

### Defining Useful Functions ### 

In [4]:
x_mod_starts = [-1950, -950, 50, 950, 1050]
# y_layer_starts = [8550, 8631.6, 9730, 9811.6, 9893.2, 9974.8, 10056.4, 10138] # 6 layer 
y_layer_starts = [8550, 8631.6, 9893.2, 9974.8, 10056.4, 10138] # 4 layer
n_floors = 2
z_mod_starts = [7000, 8000, 9000, 10000]
# z_wall_starts = [10900, 10981.6, 11063.2, 11144.8, 11226.4, 11308] # 6 layer
z_wall_starts = [10900, 10981.6, 11063.2, 11144.8, 11226.4, 11308] # 4 layer
# back_top =  y_layer_starts[4] # 6 layer
back_top =  y_layer_starts[2] # 4 layer
beam_dim = 10
mod_length = 900
front_wall_starts = [6895.8,6997.4]
steel_height=3 # cm
Box_IP_Depth=8547 # cm   
c = 29.979

wall_floor1 = (7000,8633.2)
wall_floor2 = (7000,8551.6)
wallMid1 = 6896.6
wallMid2 = 6998.2
floorMid1 = 8550.8
floorMid2 = 8632.4

IP = (0,0,0)

y_bottoms = [8550, 8631.6]
z_fronts = [6895.8, 6997.4]
x_lims = (-1950, 1950)
thickness = 1.6

In [5]:
    
def MakeBeams():
    """
    Returns a list with beams of the form 
    ((x_min, x_max),(y_min,y_max),(z_min, z_max))
    """
    beams = []
    for i in range(len(y_layer_starts) - 1):
        for j in range(len(x_mod_starts)):
            for k in range(len(z_mod_starts)):
                y_min = y_layer_starts[i] + thickness; y_max = y_layer_starts[i+1]
                
                x_min = x_mod_starts[j]; x_max = x_mod_starts[j] + beam_dim
                z_min = z_mod_starts[k]; z_max = z_mod_starts[k] + beam_dim
                beams.append(np.array(((x_min, x_max),(y_min,y_max),(z_min,z_max))))
                
                x_min = x_mod_starts[j]; x_max = x_mod_starts[j] + beam_dim
                z_min = z_mod_starts[k] + mod_length - beam_dim; z_max = z_mod_starts[k] + mod_length
                beams.append(np.array(((x_min, x_max),(y_min,y_max),(z_min,z_max))))

                x_min = x_mod_starts[j] + mod_length - beam_dim; x_max = x_mod_starts[j] + mod_length
                z_min = z_mod_starts[k]; z_max = z_mod_starts[k] + beam_dim
                beams.append(np.array(((x_min, x_max),(y_min,y_max),(z_min,z_max))))

                x_min = x_mod_starts[j] + mod_length - beam_dim; x_max = x_mod_starts[j] + mod_length
                z_min = z_mod_starts[k] + mod_length - beam_dim; z_max = z_mod_starts[k] + mod_length
                beams.append(np.array(((x_min, x_max),(y_min,y_max),(z_min,z_max))))
    return beams


def MakeMaterials():
    """
    Returns all the materials concatenated together
    """
    beams = MakeBeams()
    return beams


# Background #

In [13]:
data_top_dir = f"/home/owhgabri/My_GitHub/data/Reconstruction/10xCosmicBackground"
pathList=[]

for rootFile, dirs, files in os.walk(data_top_dir):
    for filename in files:
        if ".pkl" in filename:
            pathList.append(os.path.join(rootFile, filename))


print(len(pathList))

187


## Survival Rate: Vertex Level ##

In [10]:
nVertices = 0
n_surviving = 0
materials = MakeMaterials()
for dist in dists:
    counts.append(0)
for f in pathList:
    events=joblib.load(f)
    file_vertices = events["vertices"]
    for i in range(len(file_vertices)):
        vertices = file_vertices[i]
        nVertices += len(vertices)
        if len(vertices) == 0:
            continue
        for vertex in vertices:
            if vertex.y0 < y_layer_starts[2] and \
            vertex.y0 > 8500 and vertex.x0 > -2000 and \
            vertex.x0 < 2000 and vertex.z0 > 6800 and \
            vertex.z0 < 11350:
                n_surviving += 1
print("Total Vertices:", nVertices)
print("Survival Fraction:", n_surviving/nVertices)

Total Vertices: 4389
Survival Fraction: 0.40419229892914105


## Survival Rate: Event Level ##

In [14]:
nEvents = 0
n_surviving = 0
materials = MakeMaterials()
for f in pathList:
    events=joblib.load(f)
    file_vertices = events["vertices"]
    for i in range(len(file_vertices)):
        vertices = file_vertices[i]
        surviving = True
        if len(vertices) == 0:
            continue
        nEvents += 1
        for vertex in vertices:
            if not(vertex.y0 < y_layer_starts[2] and \
            vertex.y0 > 8500 and vertex.x0 > -2000 and \
            vertex.x0 < 2000 and vertex.z0 > 6800 and \
            vertex.z0 < 11350):
                surviving = False
        if surviving:
            n_surviving += 1
print("Total Events:", nEvents)
print("Survival Fraction:", n_surviving/nEvents)

Total Events: 3686
Survival Fraction: 0.38171459576776995


# Signal #

In [15]:
data_top_dir = f"/home/owhgabri/My_GitHub/data/Reconstruction/LLP"
pathList=[]

for rootFile, dirs, files in os.walk(data_top_dir):
    for filename in files:
        if ".pkl" in filename:
            pathList.append(os.path.join(rootFile, filename))


print(len(pathList))

5


## Survival Rate: Vertex Level ##

In [12]:
nVertices = 0
n_surviving = 0
materials = MakeMaterials()
for dist in dists:
    counts.append(0)
for f in pathList:
    events=joblib.load(f)
    file_vertices = events["vertices"]
    for i in range(len(file_vertices)):
        vertices = file_vertices[i]
        nVertices += len(vertices)
        if len(vertices) == 0:
            continue
        for vertex in vertices:
            if vertex.y0 < y_layer_starts[2] and \
            vertex.y0 > 8500 and vertex.x0 > -2000 and \
            vertex.x0 < 2000 and vertex.z0 > 6800 and \
            vertex.z0 < 11350:
                n_surviving += 1
print("Total Vertices:", nVertices)
print("Survival Fraction:", n_surviving/nVertices)

Total Vertices: 118152
Survival Fraction: 0.7721578983004943


## Survival Rate: Event Level ##

In [16]:
nEvents = 0
n_surviving = 0
materials = MakeMaterials()
for f in pathList:
    events=joblib.load(f)
    file_vertices = events["vertices"]
    for i in range(len(file_vertices)):
        vertices = file_vertices[i]
        surviving = True
        if len(vertices) == 0:
            continue
        nEvents += 1
        for vertex in vertices:
            if not(vertex.y0 < y_layer_starts[2] and \
            vertex.y0 > 8500 and vertex.x0 > -2000 and \
            vertex.x0 < 2000 and vertex.z0 > 6800 and \
            vertex.z0 < 11350):
                surviving = False
        if surviving:
            n_surviving += 1
print("Total Events:", nEvents)
print("Survival Fraction:", n_surviving/nEvents)

Total Events: 46153
Survival Fraction: 0.6313999089983316
